<a href="https://colab.research.google.com/github/DrLSimon/colabs/blob/master/Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Insatlling Pyorch

!pip install torch
!pip install torchvision

In [1]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [28]:
#@title Define Hyperparameters

input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 2 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step 

In [3]:
#@title Downloading MNIST data

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [29]:
#@title Define model class

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_classes = num_classes
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out
  
  def clone(self):
    other = Net(self.input_size, self.hidden_size, self.num_classes).cuda()
    state_dict = self.state_dict()
    other.load_state_dict(state_dict)
    return other


In [34]:
#@title Build the model

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [35]:
#@title Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam( net.parameters(), lr=lr)
optimizer = torch.optim.SGD( net.parameters(), lr=lr)

def test(net):
  correct = 0
  total = 0
  for images,labels in test_gen:
    images = Variable(images.view(-1,28*28)).cuda()
    labels = labels.cuda()

    output = net(images)
    _, predicted = torch.max(output,1)
    correct += (predicted == labels).sum()
    total += labels.size(0)

  return (correct / total).item() 


In [36]:
#@title Training the model
from collections import deque

memory = {}
mem_size = 200
for name in net.state_dict():
  memory[name] = deque([], maxlen = mem_size)

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    state_dict = net.state_dict()
    for name in state_dict:
      params = state_dict[name].clone()
      memory[name].append(params)
    
    
    #if (i+1) % 100 == 0:
    #  print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
    #             %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

  accuracy = test(net)
  print(f'Epoch [{epoch+1}/{num_epochs}] Accuracy of the model: {accuracy:.3f} %%')

Epoch [1/2] Accuracy of the model: 0.635 %%
Epoch [2/2] Accuracy of the model: 0.724 %%


In [ ]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 98.360 %


In [37]:
#@title compute coeffs
from math import sqrt
def coeffs(n):
  dn = pow(3+sqrt(8), n)
  dn = (dn + 1/dn)/2
  b = -1
  c = -dn
  s = 1
  coef = []
  for k in range(n):
    c = b - c
    b = (k+n)*(k-n)*b / ((k+1/2.)*(k+1))
    coef.append(c*s/dn)
    s = -s
  return coef

def compute_deltas(memory):
  deltas = {}
  for name, history in memory.items():
    history = list(history)
    deltas[name]=[history[0]]
    prev = history[0]
    for curr in history[1:]:
      deltas[name].append(curr - prev)
      prev = curr
  return deltas

def refine_net(net, memory):
  deltas = compute_deltas(memory)
  state_dict = net.state_dict()
  cs = coeffs(mem_size-1)
  for name in deltas:
    delta4name = deltas[name]
    val = delta4name[0]
    for c,d in zip(cs,delta4name[1:]):
      val += d*c
    state_dict[name] = val
  net.load_state_dict(state_dict)
fine_net = Net(input_size, hidden_size, num_classes).cuda()
refine_net(fine_net, memory)
#fine_net = net.cpu().clone().cuda()
#net.cuda()
print(test(net), test(fine_net))

0.7240999937057495 0.7234999537467957


In [ ]:
#@title Training the model
#@title Build the model

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

#@title Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

cnt = 0
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    cnt +=1
    ##images = Variable(images.view(-1,28*28)).cuda()
    ##labels = Variable(labels).cuda()
    ##
    ##optimizer.zero_grad()
    ##outputs = net(images)
    ##loss = loss_function(outputs, labels)
    ##loss.backward()
    ##optimizer.step()
    ##
    ##if (i+1) % 100 == 0:
    ##  print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
    ##             %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

print(cnt)

KeyboardInterrupt: ignored

In [ ]:
print(num_epochs*len(train_gen))
for params in net.parameters():
  test = torch.empty((200,*(params.shape)))
  print(test.shape)

12000
torch.Size([200, 500, 784])
torch.Size([200, 500])
torch.Size([200, 10, 500])
torch.Size([200, 10])
